In [ ]:
import pandas as pd
df=pd.read_csv('../processed/dataset.csv')

In [2]:
df.head()

,title_text,class
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [3]:
df.shape

(67050, 2)

In [4]:
df=df.dropna(subset=['title_text'])


In [5]:
df.shape

(67038, 2)

In [6]:
import nltk
from nltk.corpus import stopwords

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pratikshawadibhasme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer


In [10]:
ps = PorterStemmer()

def text_processing(text):
    text = str(text)  
    token = WordPunctTokenizer()
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(r'[0-9]', ' ', text)
    text = [
        ps.stem(word) 
        for word in token.tokenize(text.lower()) 
        if word not in stop_words
    ]
    return " ".join(text)

In [ ]:
processed_text=[]
for i in range(0,len(df['title_text'])):
    processed_text.append(text_processing(df['title_text'].iloc[i]))

In [21]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 26.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 29.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [22]:
len(processed_text)

67038

In [23]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfidf=TfidfVectorizer(strip_accents=None, lowercase=False,preprocessor=None, tokenizer=None,use_idf=True,norm='l2',smooth_idf=True,ngram_range=(1,3))

In [24]:
X=tfidf.fit_transform(processed_text)

In [25]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28021420 stored elements and shape (67038, 10443950)>
  Coords	Values
  (0, 1151578)	0.07893905891708615
  (0, 3276568)	0.020768084628757642
  (0, 5216863)	0.01898712331049854
  (0, 7542564)	0.11567666750909393
  (0, 3377855)	0.017966966881578738
  (0, 3349507)	0.07656268403603607
  (0, 8087110)	0.01824177215264175
  (0, 9999811)	0.016018742457325087
  (0, 7651707)	0.006321940780585613
  (0, 3972269)	0.010127176939872034
  (0, 1884987)	0.03129193829508377
  (0, 3148862)	0.017566911345729776
  (0, 1863000)	0.020468231461176673
  (0, 9905421)	0.03580323317950536
  (0, 5733215)	0.008528349452119078
  (0, 4191586)	0.013385257248991537
  (0, 3072483)	0.01705776708381854
  (0, 5871623)	0.022035766247083075
  (0, 2295892)	0.04520550933255132
  (0, 6518275)	0.04413143243694596
  (0, 9052495)	0.09388824836621731
  (0, 2193613)	0.03457979047671235
  (0, 1216779)	0.014277662583996348
  (0, 8886717)	0.022390159511704297
  (0, 9742394)	0

In [26]:
y=df['class']

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [32]:
!pip3 install xgboost


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [34]:
from xgboost import XGBClassifier
xgb=XGBClassifier()

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
preds = xgb.predict(X_test)
score = accuracy_score(y_test,preds)
print("Accuracy: ", score)

In [ ]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(xgb,open('model.pkl','wb'))